In [1]:
# Parameters 
project = 'PowerBIDemo'
project_path = f'../src/{project}'

In [2]:
# Libraries
import pyfabricops as pf
pf.enable_notebook_logging()
pf.set_auth_provider('env') 

import json

In [3]:
# Relative paths
branch = pf.get_current_branch() 
print(branch)

branches_path = '../branches.json'

workspace_suffix = pf.get_workspace_suffix(path=branches_path)  
print(workspace_suffix)

dev
-DEV


In [4]:
workspace_name = project + workspace_suffix

In [5]:
# Retrienving the workspace_id from config.json in the project_path for better performance
config_path = f'{project_path}/config.json'
with open(config_path, 'r') as f:
  config_content = json.load(f)
  config = config_content[branch]  
  workspace_id = config[project]['workspace_config']['workspace_id'] 
print(f'Workspace ID: {workspace_id}')

Workspace ID: 176fdb74-7b5b-455e-b6cf-4779b0a4c6ac


In [8]:
# Deploy the folders first.
pf.deploy_folders(
    workspace=workspace_id, 
    project_path=project_path,
    branches_path=branches_path,
  )  

pyfabricops._folders INFO: Found 2 folders containing Fabric artifacts
pyfabricops._folders WARNING: 409: {"requestId":"7da0083f-8433-4170-b01c-88e0724eb380","errorCode":"FolderDisplayNameAlreadyInUse","message":"Folder display name is already used."}.
pyfabricops._folders WARNING: 409: {"requestId":"ad7ff3d5-b5fb-4abd-b6e9-45bb2987f7c9","errorCode":"FolderDisplayNameAlreadyInUse","message":"Folder display name is already used."}.
pyfabricops._folders INFO: Created folders for workspace 176fdb74-7b5b-455e-b6cf-4779b0a4c6ac.
pyfabricops._folders INFO: Updated folders config for workspace "PowerBIDemo" in branch "dev"
pyfabricops._folders INFO: Folders configuration successfully written to ../src/PowerBIDemo\config.json.


In [10]:
# Deploy the semantic model
pf.deploy_semantic_model(
    workspace=workspace_id, 
    display_name='Sales',
    project_path=project_path,
    branches_path=branches_path,
)

pyfabricops._semantic_models WARNING: No valid semantic models found in workspace '176fdb74-7b5b-455e-b6cf-4779b0a4c6ac'.
pyfabricops._semantic_models INFO: Creating new semantic model: Sales
pyfabricops._semantic_models INFO: Successfully created semantic model 'Sales'
pyfabricops._semantic_models WARNING: No valid semantic models found in workspace '176fdb74-7b5b-455e-b6cf-4779b0a4c6ac'.


In [11]:
# Retrieving the semantic_model_details
pf.export_semantic_model(
    workspace=workspace_id, 
    semantic_model='Sales',
    project_path=project_path,
    branches_path=branches_path,
)

pyfabricops._semantic_models INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...
pyfabricops._utils INFO: Item definition unpacked to ../src/PowerBIDemo\workspace\Main/Sales.SemanticModel


In [27]:
import re

with open(f'{project_path}/workspace/Main/Sales.Report/definition.pbir', 'r') as f:
    report_definition = json.load(f)

dataset_reference = report_definition['datasetReference']

if 'byPath' in dataset_reference:
    dataset_path = dataset_reference['byPath']['path'] 
    dataset_name = dataset_path.split('/')[-1].split('.SemanticModel')[0]

elif 'byConnection' in dataset_reference:
    text_to_search = dataset_reference['byConnection']['connectionString']
    # Capture the value after "initial catalog="
    match = re.search(r'initial catalog=([^;]+)', text_to_search)
    if match:
      dataset_name = match.group(1)

print(dataset_name)

Sales


In [28]:
# Search for the semantic model in the config.json
with open(config_path, 'r') as f:
    config_content = json.load(f)
    config = config_content[branch]
    semantic_model_id = config[project]['semantic_models'][dataset_name]['id'] 
print(f'Semantic Model ID: {semantic_model_id}')    

Semantic Model ID: 6dd8bff3-110f-4517-af8e-2ecbad8a7159


In [29]:
# Replace the definition.pbir with the updated template
with open(f'../template_report_definition.pbir', 'r', encoding='utf-8') as f:
    report_definition_template = f.read()

report_definition_updated = report_definition_template.replace('{workspace_name}', workspace_name)
report_definition_updated = report_definition_updated.replace('{semantic_model_name}', dataset_name)
report_definition_updated = report_definition_updated.replace('{semantic_model_id}', semantic_model_id)
print(report_definition_updated)    

with open(f'{project_path}/workspace/Main/Sales.Report/definition.pbir', 'w', encoding='utf-8') as f:
    f.write(report_definition_updated)


{
  "$schema": "https://developer.microsoft.com/json-schemas/fabric/item/report/definitionProperties/1.0.0/schema.json",
  "version": "4.0",
  "datasetReference": {
    "byConnection": {
      "connectionString": "Data Source=powerbi://api.powerbi.com/v1.0/myorg/PowerBIDemo-DEV;initial catalog=Sales;integrated security=ClaimsToken",
      "pbiServiceModelId": null,
      "pbiModelVirtualServerName": "sobe_wowvirtualserver",
      "pbiModelDatabaseName": "6dd8bff3-110f-4517-af8e-2ecbad8a7159",
      "name": "EntityDataSource",
      "connectionType": "pbiServiceXmlaStyleLive"
    }
  }
}


In [30]:
# Deploy the report
pf.deploy_report(
    workspace=workspace_id, 
    display_name='Sales',
    project_path=project_path,
    branches_path=branches_path,
)


pyfabricops._reports INFO: Creating new report: Sales
pyfabricops._reports INFO: Successfully created report 'Sales'


{'id': 'ec2c4cce-50d5-4735-9c97-31beab5da7d6',
 'type': 'Report',
 'displayName': 'Sales',
 'description': '',
 'workspaceId': '176fdb74-7b5b-455e-b6cf-4779b0a4c6ac',
 'folderId': 'c891b756-ecfc-4c4c-8d87-53fd5780caa7'}

In [31]:
# Retornar o report_definition original para o arquivo .pbir
with open(f'{project_path}/workspace/Main/Sales.Report/definition.pbir', 'w', encoding='utf-8') as f:
  json.dump(report_definition, f, indent=2)

In [40]:
# For each report, we will define the report attached to the semantic model and deploy it.
import glob 
import re

for report_path in glob.glob(f'{project_path}/**/*.Report', recursive=True):
	print(f'Processing report: {report_path}')
	report_name = report_path.replace('\\', '/').split('/')[-1].split('.Report')[0] 
	print(f'Deploying report: {report_name}')

	with open(f'{report_path}/definition.pbir', 'r') as f:
		report_definition = json.load(f)

	dataset_reference = report_definition['datasetReference']

	if 'byPath' in dataset_reference:
		dataset_path = dataset_reference['byPath']['path'] 
		dataset_name = dataset_path.split('/')[-1].split('.SemanticModel')[0]

	elif 'byConnection' in dataset_reference:
		text_to_search = dataset_reference['byConnection']['connectionString']
		# Capture the value after "initial catalog="
		match = re.search(r'initial catalog=([^;]+)', text_to_search)
		if match:
			dataset_name = match.group(1)

	print(f'Semantic model: {dataset_name}')
	
	# Search for the semantic model in the config.json
	with open(config_path, 'r') as f:
		config_content = json.load(f)
	config = config_content[branch]
	semantic_model_id = config[project]['semantic_models'][dataset_name]['id'] 
	print(f'Semantic Model ID: {semantic_model_id}')  
	
	# Replace the definition.pbir with the updated template
	with open(f'../template_report_definition.pbir', 'r', encoding='utf-8') as f:
		report_definition_template = f.read()

	report_definition_updated = report_definition_template.replace('{workspace_name}', workspace_name)
	report_definition_updated = report_definition_updated.replace('{semantic_model_name}', dataset_name)
	report_definition_updated = report_definition_updated.replace('{semantic_model_id}', semantic_model_id)
	# print(report_definition_updated)    

	with open(f'{report_path}/definition.pbir', 'w', encoding='utf-8') as f:
		f.write(report_definition_updated)
	
	# Deploy the report
	pf.deploy_report(
		workspace=workspace_id, 
		display_name=report_name,
		project_path=project_path,
		branches_path=branches_path,
	)
	
	# Write back the original report_definition for the definition.pbir
	with open(f'{report_path}/definition.pbir', 'w', encoding='utf-8') as f:
		json.dump(report_definition, f, indent=2)


Processing report: ../src/PowerBIDemo\workspace\Customers\Customers.Report
Deploying report: Customers
Semantic model: Sales
Semantic Model ID: 6dd8bff3-110f-4517-af8e-2ecbad8a7159


pyfabricops._reports INFO: Report 'Customers' already exists, updating...
pyfabricops._reports INFO: Successfully updated report 'Customers'


Processing report: ../src/PowerBIDemo\workspace\Main\Sales.Report
Deploying report: Sales
Semantic model: Sales
Semantic Model ID: 6dd8bff3-110f-4517-af8e-2ecbad8a7159


pyfabricops._reports INFO: Report 'Sales' already exists, updating...
pyfabricops._reports INFO: Successfully updated report 'Sales'


In [41]:
# Extract the reports and dataflows from the workspace to update config.json
pf.export_all_reports(
	workspace=workspace_id, 
	project_path=project_path,
	branches_path=branches_path,
)

pf.export_all_dataflows_gen1(
    workspace=workspace_id, 
	project_path=project_path,
	branches_path=branches_path,
)

pyfabricops._reports INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...
pyfabricops._utils INFO: Item definition unpacked to ../src/PowerBIDemo\workspace\Customers/Customers.Report
pyfabricops._reports INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...
pyfabricops._utils INFO: Item definition unpacked to ../src/PowerBIDemo\workspace\Main/Sales.Report
pyfabricops._dataflows_gen1 INFO: Exported dataflow Calendar to ../src/PowerBIDemo\workspace\Calendar.Dataflow.
pyfabricops._dataflows_gen1 INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...


In [42]:
import subprocess

message = f'feat: {project}: Development done for the semantic model and reports'

# Git commit and checkout to new branch to start development
# Add  the project path to git
subprocess.run(['git', 'add', project_path])

# Commit the changes
subprocess.run(['git', 'commit', '-m', message]) 

CompletedProcess(args=['git', 'commit', '-m', 'feat: PowerBIDemo: Development done for the semantic model and reports'], returncode=0)